In [1]:
pip install pulp

     |████████████████████████████████| 40.6MB 91kB/s 


# Import basic libraries

In [6]:
from collections import defaultdict
from pprint import pprint
import random

from pulp import *

# This function generates possible trips

In [7]:
def generate_trips(route_names, start_hour, end_hour):
  results = []
  for route_name in route_names:
    for hour in range(start_hour, end_hour):
      results.append((route_name, hour))
  return results

# Generates duties

In [8]:
def generate_duties(num_duties, trips):

  possible_hours = {t[1] for t in trips}
  possible_routes_for_hour = defaultdict(list)
  for t in trips:
    possible_routes_for_hour[t[1]].append(t[0])

  duties = []

  for _ in range(num_duties):
    num_trips = random.randrange(1, len(possible_hours) + 1)
    hours = random.sample(possible_hours, num_trips)
    duty = []
    for hour in sorted(hours):
      route_name = random.choice(possible_routes_for_hour[hour])
      duty.append((route_name, hour))
    duties.append(duty)

  return duties


# Calculate cost for each duty

In [9]:
def cost(duty):

  # Rule 1 & 2
  pay = max(len(duty), 8)
  # Rule 3
  if len(duty) > 8:
    pay += 0.5 * (len(duty) - 8)

  # Rule 4: If there's no break, say pay to a very large value, since it violates a
  # basic labor law.
  if len(duty) > 4:
    first_hour = duty[0][1]
    last_hour = duty[-1][1]
    if last_hour - first_hour + 1 == len(duty):
      # No break
      pay = 1e6

  return pay

# Solver for our problem

In [10]:
def solve(duties, trips):
  problem = LpProblem('driver_scheduling', LpMinimize)
  variables = []
  costs = []
  # Data structure to generate constraints for each trip.
  variables_for_trip = {trip: [] for trip in trips}

  duties = duties + [[trip] for trip in trips]

  # Gather up variables and costs
  for i, duty in enumerate(duties):
    # Create a binary variable (the value can be only 0 or 1)
    x = LpVariable('x{}'.format(i + 1), 0, 1, LpBinary)
    variables.append(x)
    costs.append(cost(duty))
    for trip in duty:
      variables_for_trip[trip].append(x)

  # Create the objective function. lpDot is shorthand for
  # c * x for (c, x) in zip(costs, variables)
  problem += lpDot(costs, variables)

  # Create constraints that for each trip, exactly one x from the duties
  # including it must be 1.
  for xs in variables_for_trip.values():
    problem += lpSum(xs) == 1

  # Pulp gives a very nice string representation of the problem when printed.
  print(problem)
  status = problem.solve()
  print(LpStatus[status])

  # We have a solution, now look at the values of xs to determine which duties
  # to use. Sum the cost for each used duty.
  solution = []
  total_cost = 0
  for i, x in enumerate(variables):
    if x.value():
      solution.append(duties[i])
      total_cost += costs[i]

  return solution, total_cost

# Our main method

In [11]:
def main():
  routes = 'ABCD'
  start_hour = 9
  end_hour = start_hour + 12

  trips = generate_trips(routes, start_hour, end_hour)
  duties = generate_duties(1000, trips)
  solution_duties, solution_cost = solve(duties, trips)
  print("Cost: {}".format(solution_cost))
  pprint(solution_duties)

if __name__ == '__main__':
  main()

driver_scheduling:
MINIMIZE
8*x1 + 8*x10 + 8*x100 + 8*x1000 + 8*x1001 + 8*x1002 + 8*x1003 + 8*x1004 + 8*x1005 + 8*x1006 + 8*x1007 + 8*x1008 + 8*x1009 + 11.0*x101 + 8*x1010 + 8*x1011 + 8*x1012 + 8*x1013 + 8*x1014 + 8*x1015 + 8*x1016 + 8*x1017 + 8*x1018 + 8*x1019 + 12.5*x102 + 8*x1020 + 8*x1021 + 8*x1022 + 8*x1023 + 8*x1024 + 8*x1025 + 8*x1026 + 8*x1027 + 8*x1028 + 8*x1029 + 8*x103 + 8*x1030 + 8*x1031 + 8*x1032 + 8*x1033 + 8*x1034 + 8*x1035 + 8*x1036 + 8*x1037 + 8*x1038 + 8*x1039 + 8*x104 + 8*x1040 + 8*x1041 + 8*x1042 + 8*x1043 + 8*x1044 + 8*x1045 + 8*x1046 + 8*x1047 + 8*x1048 + 8*x105 + 8*x106 + 8*x107 + 8*x108 + 8*x109 + 8*x11 + 8*x110 + 8*x111 + 8*x112 + 1000000.0*x113 + 8*x114 + 8*x115 + 8*x116 + 9.5*x117 + 8*x118 + 11.0*x119 + 1000000.0*x12 + 8*x120 + 8*x121 + 11.0*x122 + 8*x123 + 1000000.0*x124 + 8*x125 + 8*x126 + 8*x127 + 8*x128 + 8*x129 + 8*x13 + 8*x130 + 8*x131 + 8*x132 + 8*x133 + 8*x134 + 8*x135 + 9.5*x136 + 11.0*x137 + 9.5*x138 + 8*x139 + 9.5*x14 + 12.5*x140 + 12.5*x141 + 8*x1